In [18]:
import time
import datetime
import pandas as pd
from environments.forex_trading import tgym

In [ ]:
import os
files=[]
for i in range(51):
    file = f"./data/split/GBPUSD/weekly/GBPUSD_2017_{i}.csv"
    if os.path.isfile(file):
        files.append(file)
    else:
        print(f'not exist: {file}')
print(files)    


In [20]:
def train(env, agent, files, if_vix = True,**kwargs):
    learning_rate = kwargs.get('learning_rate', 2 ** -15)
    batch_size = kwargs.get('batch_size', 2 ** 11 )
    gamma = kwargs.get('gamma', 0.99)
    seed = kwargs.get('seed', 312)
    total_timesteps = kwargs.get('total_timesteps', 1e6)
    net_dimension = kwargs.get('net_dimension', 2**9)
    cwd = kwargs.get('cwd','./'+str(agent))

    # env_instance = map(env, [pd.read_csv(f) for f in files])
    if agent == 'ppo':
        from stable_baselines3 import PPO
        from stable_baselines3.common.vec_env import SubprocVecEnv, DummyVecEnv

        # env_train = [x.get_sb_env for x in env_instance ]
        vector_env = [lambda:env(df=pd.read_csv(f)) for f in files]
        env_train = SubprocVecEnv(vector_env)
        model = PPO("MlpPolicy", env_train, learning_rate=learning_rate, 
                    n_steps=2048, batch_size=batch_size, ent_coef=0.0, 
                    gamma=gamma, seed=seed)
        start_time = time.time()
        s = datetime.datetime.now()
        print(f'Training start: {s}')
        model.learn(total_timesteps=total_timesteps, tb_log_name = 'ppo')
        print('Training finished!')
        model_name = "./data/models/GBPUSD-week-" + s.strftime('%Y%m%d%H%M%S')
        model.save(model_name)
        print(f'Trained model saved in {model_name}')
        print(f"trainning time: {(time.time() - start_time)}")

    else:
        raise ValueError('DRL library input is NOT supported. Please check.')

In [21]:
train(env=tgym,agent="ppo",files=files)

Training start: 2021-10-25 22:40:25.516039
Training finished!
Trained model saved in ./ppo
trainning time: 359.4131119251251


In [ ]:
run at 10:36 to check loading time.